##### Copyright 2021 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/qcvv/xeb_characterization_pipeline>"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/qcvv/xeb_characterization_pipeline.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/qcvv/xeb_characterization_pipeline.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/qcvv/xeb_characterization_pipeline.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

This notebook is a straightforward outline of how to characterize coherent error with [Cross Entropy Benchmarking (XEB)](./xeb_theory.ipynb). 

In [ ]:
try:
    import cirq
except ImportError:
    !pip install --quiet cirq
    import cirq
import numpy as np

## Set up Random Circuits

Create a library of random, two-qubit `circuits` using the `SQRT_ISWAP` gate. These library circuits will be truncated to circuits by lengths defined in  `cycle_depths`, and mixed-and-matched among all the qubit pairs on the device to be characterized.

In [ ]:
from cirq.experiments import random_quantum_circuit_generation as rqcg

RANDOM_SEED = np.random.RandomState(53)
circuit_library = rqcg.generate_library_of_2q_circuits(
    n_library_circuits=20, 
    two_qubit_gate=cirq.SQRT_ISWAP,
    random_state=RANDOM_SEED,
)
# Will truncate to these lengths
max_depth = 100
cycle_depths = np.arange(3, max_depth, 20)

## Determine the device topology

XEB can be run on all pairs from a given device topology. Below, you can supply a `device_name` if you're authenticated to run on Google QCS. This case will get the device object from the cloud endpoint and turn it into a graph of qubits. Otherwise, mock a device graph by allocating arbitrary `cirq.GridQubit`s into a graph.

In [ ]:
device_name = None  # change me!

import networkx as nx
import itertools

if device_name is not None:
    import cirq_google as cg
    sampler = cg.get_engine_sampler(device_name, gate_set_name='sqrt_iswap')
    device = cg.get_engine_device(device_name)
    qubits = sorted(device.qubits)
else:
    qubits = cirq.GridQubit.rect(3, 2, 4, 3)
    # Delete one qubit from the rectangular arangement to
    # 1) make it irregular 2) simplify simulation.
    qubits = qubits[:-1]
    noise_model = cirq.ConstantQubitNoiseModel(cirq.depolarize(5e-3))
    sampler = cirq.DensityMatrixSimulator(noise=noise_model)

# create a graph from adjacent qubits and draw it
graph = nx.Graph((q1,q2) for (q1,q2) in itertools.combinations(qubits, r=2) if q1.is_adjacent(q2))
pos = {q: (q.row, q.col) for q in qubits}
nx.draw_networkx(graph, pos=pos)

## Set up qubit pair combinations

XEB can be performed in a parallel or isolated manner. In the parallel case, multiple qubits are "active" to be tested simultaneously in the same circuit, where the isolated case has only one active pair running at any time. The following cell creates the `combs_by_layer` data structure, which defines which qubit pairs are active in parallel, and which two-qubit circuits (from `circuit_library`) will be applied to each of the active pairs.

The outer list  of `combs_by_layer` corresponds to the different layers. Each inner layer defines the `combinations` matrix, which indexes the circuit library circuits to use, and the `pairs` list, which notes which qubit pairs are active.

In [ ]:
parallel = True

if parallel:
  # automatically selects multiple active pairs per layer
  combs_by_layer = rqcg.get_random_combinations_for_device(
      n_library_circuits=len(circuit_library),
      n_combinations=10,
      device_graph=graph,
      random_state=RANDOM_SEED,
  )
else:
  # only one active pair per layer
  layer_pairs = [[pair] for pair in graph.edges]
  combs_by_layer = rqcg.get_random_combinations_for_pairs(
      n_library_circuits=len(circuit_library), 
      n_combinations=10,
      all_pairs=layer_pairs,
      random_state=RANDOM_SEED,
  )
print(len(combs_by_layer))
combs_by_layer[0]

### Visualize
Here, draw the layers' active pairs to see which qubits will be tested in which layers. The active pairs are highlighted in red, and some layers demonstrate non-overlapping qubit pairs which will be active in parallel (if using `parallel=true` above).

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

ncols = int(len(combs_by_layer)**(1/2))
nrows = ncols+1 if ncols**2 < len(combs_by_layer) else ncols
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(9,6))
for comb_layer, ax in zip(combs_by_layer, axes.reshape(-1)):
    active_qubits = np.array(comb_layer.pairs).reshape(-1)
    # highlight the active qubits in red
    colors = ['red' if q in active_qubits else 'blue' for q in graph.nodes]
    nx.draw_networkx(graph, pos=pos, node_color=colors, ax=ax)
    # highlight the active pair edges in red 
    nx.draw_networkx_edges(graph, pos=pos, edgelist=comb_layer.pairs, width=3, edge_color='red', ax=ax)
    
plt.tight_layout()

## Sample Data

The sampling function `sample_2q_xeb_circuits` zips the circuits of `circuit_library` together according to `combs_by_layer`, before sampling them and computing the sampled probabilities of getting the correct bit string `sampled_probs`. 

For layers with multiple active qubit pairs, it will combine the two-qubit circuits from `circuit_library` by placing different circuits on different active pairs. This creates larger, many-pair circuits, which operate on all the active pairs of a layer in parallel, without ever connecting the distinct pairs' circuits to each other. An inspection of one of the sampled `DataFrame`s reveals the available types of data, including the sampled probabilities `sampled_probs`.


In [ ]:
from cirq.experiments.xeb_sampling import sample_2q_xeb_circuits
sampled_df = sample_2q_xeb_circuits(
    sampler=sampler,
    circuits=circuit_library,
    cycle_depths=cycle_depths,
    combinations_by_layer=combs_by_layer,
    shuffle=np.random.RandomState(52),
    repetitions=10_000,
)
sampled_df.head()

## Benchmark Fidelities

The `benchmark_2q_xeb_fidelities` function takes each sampled `DataFrame` and computes the overall fidelity across all of the randomized circuits for each qubit pair and each of the previously defined `cycle_depth`s.

In [ ]:
from cirq.experiments.xeb_fitting import benchmark_2q_xeb_fidelities
fids = benchmark_2q_xeb_fidelities(
    sampled_df=sampled_df,
    circuits=circuit_library,
    cycle_depths=cycle_depths,
)
fids.head()

## Estimate By-Layer Fidelities

Next, use `fit_exponential_decays` on each of the fidelity datasets of `circuit_fidelities`, to estimate a per-layer fidelity, which estimates the fidelity error of the two-qubit entangling gate operation on each individual qubit pair. 

In [ ]:
from cirq.experiments.xeb_fitting import fit_exponential_decays, exponential_decay
fidelities = fit_exponential_decays(fids)
fidelities.head()

# Visualizations

Plot the fidelity decrease per layer by qubit in a heatmap to compare the error rate per layer across different qubits. This can highlight when different qubits are performing meaningfully better or worse than the others.

In [ ]:
heatmap_data = {}

for (_, _, pair), fidelity in fidelities.layer_fid.items():
    heatmap_data[pair] = 1.0 - fidelity

cirq.TwoQubitInteractionHeatmap(heatmap_data).plot();

The qubit heatmap summarizes the data quite a lot. Plotting the fidelities by cycle depth with their fit exponential decay curves demonstrates how error accumulates when circuits get more cycles. 

In [ ]:
import seaborn as sns

# Give each pair its own color
colors = sns.cubehelix_palette(n_colors=graph.number_of_edges())
colors = dict(zip(graph.edges, colors))

# Exponential reference
xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference', color='black')

# Plot each pair
def _p(fids):
    q0, q1 = fids.name
    plt.plot(fids['cycle_depth'], fids['fidelity'], 
             'o-', label=f'{q0}-{q1}', color=colors[fids.name], alpha=0.5)

fids.groupby('pair').apply(_p)

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth $d$')
plt.legend(loc='best')
plt.tight_layout()

Included is an exponential reference line, matching the original chance of depolarization of `5e-3`. These results reinforce the idea that uniform pauli deolarization causes an exponential decay in circuit fidelity by cycle. 

## Optimize `PhasedFSimGate` parameters

In a real experiment, there is likely unknown coherent error that you would like to characterize, by identifying the true parameters of the unitary gate implemented by the hardware. To do so, make the five angles of `PhasedFSimGate` free parameters in `SqrtISwapXEBOptions`, and use a classical optimizer (`characterize_phased_fsim_parameters_with_xeb_py_pair`) to find out which set of parameters best describes the data collected from the noisy simulator (or device, if this was a real experiment).

In [ ]:
import multiprocessing
pool = multiprocessing.get_context('spawn').Pool()

In [ ]:
from cirq.experiments.xeb_fitting import (
    parameterize_circuit, 
    characterize_phased_fsim_parameters_with_xeb_by_pair, 
    SqrtISwapXEBOptions,
)

# Set which angles to characterize (all)
options = SqrtISwapXEBOptions(
    characterize_theta = True,
    characterize_zeta = True,
    characterize_chi = True,
    characterize_gamma = True,
    characterize_phi = True
)
# Parameterize the sqrt(iswap)s in the circuit library
pcircuits = [parameterize_circuit(circuit, options) for circuit in circuit_library]

# Run the characterization loop
characterization_result = characterize_phased_fsim_parameters_with_xeb_by_pair(
    sampled_df,
    pcircuits,
    cycle_depths,
    options,
    pool=pool,
    # ease tolerance so it converges faster:
    fatol=1e-2, 
    xatol=1e-2
)

The fitting procedure finds the parameter values that are most likely to explain the data for the two-qubit unitary gate applied to each qubit pair...

In [ ]:
characterization_result.final_params

The characterization also refits the circuit fidelities by cycle, to estimate circuit performance as if the gates were performing correctly (with the correct parameters instead of those observed). 

In [ ]:
characterization_result.fidelities_df.head()

The `before_and_after_characterization` function collects the before fidelities, `circuit_fidelities`, and the after fidelities, `characterization_results` into the same data frame, by row, suffixing the data by source with `_0` and `_c` respectively. This may make it easier to plot the data by row. 

In [ ]:
from cirq.experiments.xeb_fitting import before_and_after_characterization
before_after_df = before_and_after_characterization(fids, characterization_result)
before_after_df.head()

Finally, plot the original fidelities alongside the refit fidelities, to analyze how the refitting procedure was able to identify coherent error. 

In [ ]:
for i, row in before_after_df.iterrows():
    plt.axhline(1, color='grey', ls='--')
    plt.plot(row['cycle_depths_0'], row['fidelities_0'], '*', color='red')
    plt.plot(row['cycle_depths_c'], row['fidelities_c'], 'o', color='blue')

    xx = np.linspace(0, np.max(row['cycle_depths_0']))
    plt.plot(xx, exponential_decay(xx, a=row['a_0'], layer_fid=row['layer_fid_0']), color='red', label='Original Fidelities')
    plt.plot(xx, exponential_decay(xx, a=row['a_c'], layer_fid=row['layer_fid_c']), color='blue', label='Refit Fidelities')
    
    plt.xlabel('Cycle Depth')
    plt.ylabel('Fidelity')

# make the legend labels unique
handles, labels = plt.gca().get_legend_handles_labels()
legend_items = dict(zip(labels, handles))
plt.legend(legend_items.values(), legend_items.keys(), loc='best')
plt.tight_layout()
plt.show()

The refit fidelities are almost the same as the original fidelities in this case, but are a bit better in some cases. The characterization procedure was able to identify some coherent error, but was not able to refit for all of the _incoherent_ error introduced by the depolarization. For more information on how XEB interacts with error, see [Coherent vs Incoherent Error with XEB](./coherent_vs_incoherent_xeb). 